## Codigo que permite conectar Colab con Google Drive
Este codigo fué utilizado para aprobechar los recursos entregados por Google para asi poder realizar los entrenamientos, ya que no fué posible realizarlos en los computadores que se tenian disponibles.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Cross Validation con Sklearn

### Redimensionar imagenes a 64x64
En este codigo se recorren las carpetas hasta llegar a los frames que se le reduce la dimension que tenia a 64x64 pixel, esto debido a que al usar la dimension 128x128 y pasar las imagenes vectorizadas a variable provocaba que el disco duro colapsara por un prolongado tiempo evitando que se pueda llevar a cabo este paso.

In [ ]:
import os
import cv2

cont = 0
width = 64
height = 64
dim = (width, height)

for FoldNumber in range(5):
    
    for NumberFolder in range(11):
        
        #print(NumberFolder)
        
        for NumberSample in range(64): # revisa sample por sample
            
            ListSample = os.listdir("Folders/Fold%d/%d/" % ((FoldNumber+1),NumberFolder))
            ListFrame = os.listdir("Folders/Fold%d/%d/%s/" % ((FoldNumber+1),NumberFolder,ListSample[NumberSample]))
            for NumberFrame in range(20):
                buffer = ("Folders/Fold%d/%d/%s/%s" % ((FoldNumber+1),NumberFolder,ListSample[NumberSample],ListFrame[NumberFrame]))
                buffer2 = ("FoldersRedim/Fold%d/%d/%s/" % ((FoldNumber+1),NumberFolder,ListSample[NumberSample]))
                buffer3 = ("FoldersRedim/Fold%d/%d/%s/%s" % ((FoldNumber+1),NumberFolder,ListSample[NumberSample],ListFrame[NumberFrame]))
                
                img = cv2.imread(buffer, cv2.IMREAD_UNCHANGED)
                

                # resize image
                resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
                cont = cont + 1 
                
                # Crea los directorios (Carpetas)
                if os.path.isdir(buffer2):
                    cv2.imsswrite(buffer3, resized)
                else:
                    os.makedirs(buffer2)
                    cv2.imwrite(buffer3, resized)

### Reduccion de muestras a la mitad
Debido a la necesidad de bajar la cantidad de proceso realizado por el disco duro se vio la necesidad de disminir los datos a procesar, para esto se usa el codigo siguiente que de manera random mueve muestras de una carpeta a otra nueva para reducirlas a la mitad (en equilibrio, como todo debe ser -Thanos)

In [ ]:
import shutil
import os
import random
# Se crea el la carpeta ThanosFolders si no está
try:
    os.stat("ThanosFolders")  # Revisa si está el directorio
except:
    os.mkdir("ThanosFolders") # Si no está lo crea 
# Se crean los 5 Fold
for i in range(5):
    try:
        os.stat("ThanosFolders/Fold%d" % (i+1))
    except:
        os.mkdir("ThanosFolders/Fold%d" % (i+1))
# Se crean carpetas fold/Numeros
for j in range(5):
    for i in range(11):
        try:
            os.stat("ThanosFolders/Fold%d/%d" % ((j+1), i))
        except:
            os.mkdir("ThanosFolders/Fold%d/%d" % ((j+1), i))


ShortName = ["BF", "GG", "JB", "JM"]

for FoldNumber in range(5):
  for Number in range(11):
    for ToMove in range(32):
      FilesList = os.listdir("Folders/Fold%d/%d/" % ((FoldNumber+1), Number))
      
      RandomFile = random.choice(FilesList)
      
      shutil.move(RandomFile, 
                  "ThanosFolders/Fold%d/%d/" % ((FoldNumber + 1), Number, File))


### Pasar de 20 iamgenes .jpg a .csv 

Para generar los csv se juntaron las 20 imagenes de manera tridimencional, es decir, a cada imagen (64x64) se le agregó otra detras (64x64x2), luego se le agregó otra atras de las dos adjuntadas (quedando 64x64x3), hasta que se llegó a la dimension 64x64x20, todo esto usando la linea de codigo:
* ImgStack = np.stack((img1, img2, img3, img4, img5, img6, img7, img8, img9, img10, img11, img12, img13, img14, img15, img16, img17, img18, img19, img20), axis = -1)

Luego para almacenar las imagenes (Matrices) se guardaron diractamente a un csv dentro de una carpeta que los reunió todos juntos para luego poder ser consolidados en uno solo y asi poder tener la base de datos lista para pasarla a alguna variable como "X" e "y"




In [ ]:
import cv2
import numpy as np # Imagen a Matriz
import os
import pandas as pd # Matriz a .csv

# Creacion de nuevo directorio que contendrá solo .csv
try:
    os.stat("SamplesCSV")  # Revisa si está el directorio
except:
    os.mkdir("SamplesCSV") # Si no está lo crea 

# Se crean los 5 Fold
#for i in range(5):
#    try:
#        os.stat("FoldsCSV/Fold%d" % (i+1))
#    except:
#        os.mkdir("FoldsCSV/Fold%d" % (i+1))

ShortName = ["BF", "GG", "JB", "JM"]

for FoldNumber in range(5):
    for Number in range(11):
        NumberName = 0 # contador de samples revisados
        for SampleNumber in range(320): # revisa sample por sample
            
            if SampleNumber == 80 or SampleNumber == 160 or SampleNumber == 240: # para recorrer ShortName[NumberName]
                NumberName = NumberName + 1
            if (os.path.isdir("Folders/Fold%d/%d/Sample-%s-%02d%03d" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1)))): # lo siguiente se realiza solo si el directorio se encuentra
                FrameNumber = 1
                
                # Se pasa de 20 Imagenes.jpg a 20 Matrices 2D
                for Frames in range(1, 21): # recorre los 20 frames partiendo con Frames=1
                    
                    while not (os.path.isfile("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber))):
                        FrameNumber += 1 # si no existe la imagen Frame# se busca Frame(#+1) 
                    
                    if Frames == 1:
                        img1 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 2:
                        img2 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 3:
                        img3 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 4:
                        img4 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 5:
                        img5 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 6:
                        img6 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 7:
                        img7 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 8:
                        img8 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 9:
                        img9 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 10:
                        img10 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 11:
                        img11 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 12:
                        img12 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 13:
                        img13 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 14:
                        img14 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 15:
                        img15 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 16:
                        img16 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 17:
                        img17 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 18:
                        img18 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 19:
                        img19 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)
                    elif Frames == 20:
                        img20 = cv2.imread(("Folders/Fold%d/%d/Sample-%s-%02d%03d/Frame%d.jpg" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1), FrameNumber)), cv2.IMREAD_UNCHANGED)

                    FrameNumber += 1
                        
                # 20 Matriz 2D a una Matriz 3D
                ImgStack = np.stack((img1, img2, img3, img4, img5, img6, img7, img8, img9, img10, img11, img12, img13, img14, img15, img16, img17, img18, img19, img20), axis = -1)
                
                # Se guarda la Matriz 3D como .csv
                ImgStack.tofile("SamplesCSV/Sample-%s-%02d%03d.csv" % (ShortName[NumberName], Number, (SampleNumber+1)), sep=',', format='%d')

print(ImgStack.shape)



(128, 128, 20)


### Revisa si hay carpetas samples con menos de 20 imagenes
Si las hay, imprime las encontradas. Esto se utilizó para revisar algunos salmples que quedaron defectuosos en los pasos iniciales (Cortar videos largos a videos de 2 segundos, o generacion de frames a partir de los videos cortos)

In [ ]:
ShortName = ["BF", "GG", "JB", "JM"]

for FoldNumber in range(5):
    for Number in range(11):
        NumberName = 0 # contador de samples revisados
        for SampleNumber in range(320): # revisa sample por sample
            
            if SampleNumber == 80 or SampleNumber == 160 or SampleNumber == 240: # para recorrer ShortName[NumberName]
                NumberName = NumberName + 1
            if (os.path.isdir("Folders/Fold%d/%d/Sample-%s-%02d%03d" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1)))): # lo siguiente se realiza solo si el directorio se encuentra
                FrameNumber = 1
                    
                FilesList = os.listdir("Folders/Fold%d/%d/Sample-%s-%02d%03d" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1)))
                FilesCount = len(FilesList) # cantidad de archivos en carpeta samples
                if FilesCount < 20:
                    print("Folders/Fold%d/%d/Sample-%s-%02d%03d" % ((FoldNumber+1), Number, ShortName[NumberName], Number, (SampleNumber+1)))

Folders/Fold4/3/Sample-BF-03056


### Consolidación de archivos .csv
Para consolidar los archivos .csv se recurrió a la consola de comandos (cmd) usando el siguiente comando en el directorio donde se encontraban todos los csv juntos:

**copy *.csv X.csv**

Lo que hace es copiar todos los archivos con la extencion .csv y los pega (consolidando) en el archivo X.csv creado por el mismo comando

### Creación de y.csv
La creación de y.csv al ser mas simple se hizo a mano con Excel siguiendo el orden de todos los Samples-XX-00000.csv, dejando una sola columna con los numeros correspondientes (0 repetido 80 veces hacia abajo, 1 repetido 80 veces hacia abajo, ..., 10 repetido 80 veces hacia abajo. Todo esto repetido 4 veces (80 * 11 * 4 = 3520 Samples))

### Pasar de X.csv a variable
Para intentar tener la variable "X" conteniendo todas la imagenes y asi dividirlas con la funcion de sklearn:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33, random_state=42)
se usó el siguiente código, que resultó ser demasiado proceso para un disco duro convencional HDD.

In [ ]:
###### TESTEO  1 ######
# NO EJECUTAR!!!!
import csv

results = []
with open("C:/Users/Joselmq/Documents/Sistemas Inteligentes/Imagenes/y.csv") as csvfile:
    reader = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
#    for row in reader: # each row is a list
#        results.append(row)


###### TESTEO 2 ######
#NO EJECUTAR!!!!
from sklearn.model_selection import train_test_split
import pandas as pd

#X = pd.read_csv("https://drive.google.com/file/d/1TBSXTgqIh8ZdLnAlnKZ78llabH0TpnAp/view?usp=sharing")
#y = pd.read_csv("https://drive.google.com/file/d/1XobxzrM6qzwpRevxvnmFhpd_yg4fQshj/view?usp=sharing")


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6)

## Regresión Logística
Primero, para este código, por cosas de la libreria, se requería de que la base de datos estuviera en una sola carpeta (en este caso ubica en el drive), que los archivos estuvieran en formato imagen y en escala de grises, lo cual eran fundamentales para el entrenamiento, pero por problemas de tamaño del arreglo no pudo llevarse a cabo un entrenamiento exitoso.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

XY_dategen = ImageDataGenerator()
XY_generator = XY_dategen.flow_from_directory(
    #dependera de quien vea la direcion
    #jose
    #directory=r"/content/gdrive/My Drive/2019/Sistemas Inteligentes/Proyecto Air Writing Grupo 4/Entrega 2/DataSet/1Fold",
    #gabriel
    directory=r"/content/gdrive/My Drive/Proyecto Air Writing Grupo 4/Entrega 2/DataSet/1Fold",
    target_size=(1280, 64), color_mode="grayscale", batch_size=32, class_mode="categorical", shuffle=True, seed=42)
X, Y = XY_generator.next()  #or next(train_generator)
# from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2 ) # training to testing ratio is 0.8:0.2
# split the data with 50% in each set
#X1, X2, y1, y2

# fit the model on one set of data
model = LogisticRegression(C = 10**3, solver='newton-cg', multi_class='multinomial')
#model.add(Flatten())

model.fit(X_train, Y_train)

# evaluate the model on the second set of data
Y_test_model = model.predict(X_test)
accuracy_score(Y_test, Y_test_model)

print (X_train.shape, Y_train.shape)
print (X_test.shape, Y_test.shape)

Found 3520 images belonging to 11 classes.
(25, 1280, 64, 1) (25, 11)
(7, 1280, 64, 1) (7, 11)


### <<<<<<<<<<<<<<<< Hasta acá se llegó con Sklearn >>>>>>>>>>>>>>>>
    (debido a errodes de falta de memoria al cargar datos)

In [ ]:
y2_model = model.fit(X1, y1).predict(X2)
y1_model = model.fit(X2, y2).predict(X1)
accuracy_score(y1, y1_model), accuracy_score(y2, y2_model)

### Realización de la validación cruzada
En esta parte se generan 5 particiones del dataset y se hace el entrenamiento 5 veces, cada vez usando un batch distinto para el validate.

In [ ]:
from sklearn.cross_validation import cross_val_score
cross_val_score(model, X, y, cv=5)

### Metodos para mejorar el entrenamiento



In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))
  
import numpy as np

def make_data(N, err=1.0, rseed=1):
    # randomly sample the data
    rng = np.random.RandomState(rseed)
    X = rng.rand(N, 1) ** 2
    y = 10 - 1. / (X.ravel() + 0.1)
    if err > 0:
        y += err * rng.randn(N)
    return X, y

X, y = make_data(40)

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # plot formatting

X_test = np.linspace(-0.1, 1.1, 500)[:, None]

plt.scatter(X.ravel(), y, color='black')
axis = plt.axis()
for degree in [1, 3, 5]:
    y_test = PolynomialRegression(degree).fit(X, y).predict(X_test)
    plt.plot(X_test.ravel(), y_test, label='degree={0}'.format(degree))
plt.xlim(-0.1, 1.0)
plt.ylim(-2, 12)
plt.legend(loc='best');


############################################


from sklearn.learning_curve import validation_curve
degree = np.arange(0, 21)
train_score, val_score = validation_curve(PolynomialRegression(), X, y,
                                          'polynomialfeatures__degree', degree, cv=7)

plt.plot(degree, np.median(train_score, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');


############################################


plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = PolynomialRegression(3).fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test);
plt.axis(lim);


############################################


degree = np.arange(21)
train_score2, val_score2 = validation_curve(PolynomialRegression(), X2, y2,
                                            'polynomialfeatures__degree', degree, cv=7)

plt.plot(degree, np.median(train_score2, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score2, 1), color='red', label='validation score')
plt.plot(degree, np.median(train_score, 1), color='blue', alpha=0.3, linestyle='dashed')
plt.plot(degree, np.median(val_score, 1), color='red', alpha=0.3, linestyle='dashed')
plt.legend(loc='lower center')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');


############################################


from sklearn.learning_curve import learning_curve

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for i, degree in enumerate([2, 9]):
    N, train_lc, val_lc = learning_curve(PolynomialRegression(degree),
                                         X, y, cv=7,
                                         train_sizes=np.linspace(0.3, 1, 25))

    ax[i].plot(N, np.mean(train_lc, 1), color='blue', label='training score')
    ax[i].plot(N, np.mean(val_lc, 1), color='red', label='validation score')
    ax[i].hlines(np.mean([train_lc[-1], val_lc[-1]]), N[0], N[-1],
                 color='gray', linestyle='dashed')

    ax[i].set_ylim(0, 1)
    ax[i].set_xlim(N[0], N[-1])
    ax[i].set_xlabel('training size')
    ax[i].set_ylabel('score')
    ax[i].set_title('degree = {0}'.format(degree), size=14)
    ax[i].legend(loc='best')



# Método Keras (Test, Train, Valid)




### Crear carpeta CVFolders

Manualmete se creó la carpeta CVFolders y dentro de ella se crearon las carpetas Test, Train y Valid. Se pegó el contenido de los  3 primeros Folds para Train (60%), el Fold4 para Valid (20%) y el ultimo Fold5 para Test (20%)

### Juntar las 20 imágenes en una de manera horizontal
En la carpeta "CrossValidationFolder" se crean tres subcarpetas (Train, Test y Valid) en las que se crean las uniones de las secuencias de imagenes de cada sample ubicado en CVFolders.

In [ ]:
import numpy as np
import PIL
from PIL import Image
import os
import re #Split


# para ordenar correctamente la lista de imagenes en el directorio
def sorted_aphanumeric(data): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)


# Arrays para recorrer directorios
TestTrainValid = ["Test", "Train", "Valid"]
ShortName = ["BF", "GG", "JB", "JM"]


# Se crea el la carpeta CrossValidationFolder si no está
try: 
    os.stat("CrossValidationFolder")
except:
    os.mkdir("CrossValidationFolder")
# Se crean las carpetas "Test", "Train" y "Valid"
for i in range(3): 
    try:
        os.stat("CrossValidationFolder/%s" % TestTrainValid[i])
    except:
        os.mkdir("CrossValidationFolder/%s" % TestTrainValid[i])
# Se crean carpetas Numeros
for j in range(3):
    for i in range(11):
        if j == 0:
            try:
                os.stat("CrossValidationFolder/%s/Test_folder" % TestTrainValid[j])
            except:
                os.mkdir("CrossValidationFolder/%s/Test_folder" % TestTrainValid[j])
        else:
            try:
                os.stat("CrossValidationFolder/%s/Class_%d" % (TestTrainValid[j], i))
            except: 
                os.mkdir("CrossValidationFolder/%s/Class_%d" % (TestTrainValid[j], i))


for Folds in range(len(TestTrainValid)):
    for Number in range(11):
        NumberName = 0
        for SampleNumber in range(320):
            if SampleNumber == 80 or SampleNumber == 160 or SampleNumber == 240: # para recorrer ShortName[NumberName]
                NumberName = NumberName + 1
            if (os.path.isdir("CVFolders/%s/%d/Sample-%s-%02d%03d" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1)))):
                
                FilesList = sorted_aphanumeric(os.listdir("CVFolders/%s/%d/Sample-%s-%02d%03d" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1)))) # Ordena archivos en directorio
                list_im = ["CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[0]),
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[1]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[2]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[3]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[4]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[5]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[6]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[7]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[8]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[9]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[10]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[11]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[12]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[13]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[14]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[15]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[16]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[17]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[18]), 
                           "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[19]),]
                
                imgs = [ PIL.Image.open(i) for i in list_im ]
                # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
                min_shape = sorted([(np.sum(i.size), i.size ) for i in imgs])[0][1]
                imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

                # save that beautiful picture
                imgs_comb = PIL.Image.fromarray( imgs_comb)
                if Folds == 0:
                    imgs_comb.save("CrossValidationFolder/%s/Test_folder/Sample-%s-%02d%03d.jpg" % (TestTrainValid[Folds], ShortName[NumberName], Number, (SampleNumber+1)))   
                else:
                    imgs_comb.save("CrossValidationFolder/%s/Class_%d/Sample-%s-%02d%03d.jpg" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1)))   

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


### Juntar las 20 imágenes en una de manera horizontal (Con reducción de muestras)
Debido a la falta de memoria y de los computadores de la plataforma Colab () se redujeron la cantidad de muestras para así poder ejecutar las muestras a las variables correspondientes a train, test y valid.




---

Error mostrado por Colab


---


Te estás quedando sin espacio en la memoria
Tu entorno de ejecución se está quedando sin espacio en la memoria. Si se sigue empleando, es posible que se produzca un error por falta de memoria y que se pierda el estado en memoria de todas las sesiones. ¿Quieres finalizar algunas sesiones para liberar memoria (dichas sesiones perderán su estado)? Actualmente se están utilizando 10.79 GB de 12.72 GB.


---



la reduccion de muestras se hizo de manera equitativa, lo que significa, que la cantidad de muestras finales es la misma por cada integrante y número, conservando tambien la proporcion de los distintos tamaños de muestras entre Train y las otras dos carpetas, usando (numero * i) para realizar el ajuste (i es 3 para Train y 1 en caso contrario). 


Primero se probó el número 20 para multiplicar "i" (20 muestras por cada personas y en cada numero), lo que no mantuvo la proporcionalidad requerida debido a que las muestras fueron repartidas aleatoriamente entre los 5 folds en pasos anteriores, por lo que en algunos numero no se llegaba a las 20 muestras por personas.


Luego se probó con el numero 10 y se procedió con los pasos posteriores, pero al pasar los datos a los Xtrain y Xtest el programa mostró falta de memoria. Despues se cambió el 10 por el 5 y el 5 por el 2 resultando en el mismo problema de no poder cargar los datos.


---
 if ((NumberName == 0 and CountToReduceSamples < (2 * i)) or (NumberName == 1 and CountToReduceSamples < (4 * i)) or (NumberName == 2 and CountToReduceSamples < (6 * i)) or (NumberName == 3 and CountToReduceSamples < (8 * i))):

 


In [ ]:
import numpy as np
import PIL
from PIL import Image
import os
import re #Split
import cv2


# para ordenar correctamente la lista de imagenes en el directorio
def sorted_aphanumeric(data): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)


# Arrays para recorrer directorios
TestTrainValid = ["Test", "Train", "Valid"]
ShortName = ["BF", "GG", "JB", "JM"]


# Se crea el la carpeta CrossValidationFolder si no está
try: 
    os.stat("CrossValidationFolderSmallRedim")
except:
    os.mkdir("CrossValidationFolderSmallRedim")
# Se crean las carpetas "Test", "Train" y "Valid"
for i in range(3): 
    try:
        os.stat("CrossValidationFolderSmallRedim/%s" % TestTrainValid[i])
    except:
        os.mkdir("CrossValidationFolderSmallRedim/%s" % TestTrainValid[i])
# Se crean carpetas Numeros
for j in range(3):
    for i in range(11):
        if j == 0:
            try:
                os.stat("CrossValidationFolderSmallRedim/%s/Test_folder" % TestTrainValid[j])
            except:
                os.mkdir("CrossValidationFolderSmallRedim/%s/Test_folder" % TestTrainValid[j])
        else:
            try:
                os.stat("CrossValidationFolderSmallRedim/%s/%d" % (TestTrainValid[j], i))
            except: 
                os.mkdir("CrossValidationFolderSmallRedim/%s/%d" % (TestTrainValid[j], i))

                

for Folds in range(len(TestTrainValid)):
    if Folds == 1: # si fold = Train   (donde hay 3 folds) (por problemas de memoria)
        i = 3 # multiplicador cantidad de muestras a pasar a CrossValidationFolderRedim
    else:
        i = 1
    for Number in range(11):
        NumberName = 0
        CountToReduceSamples = 0
        for SampleNumber in range(320):
            if SampleNumber == 80 or SampleNumber == 160 or SampleNumber == 240: # para recorrer ShortName[NumberName]
                NumberName = NumberName + 1
            # Solo extrae 2 muestras de cada persona por cada "CVFolders/(Train, Test o Valid)/(0 a 10)      (40 muestras "0": 8 muestras de Test, 8 de Train y 24 de Test)
            if ((NumberName == 0 and CountToReduceSamples < (2 * i)) or (NumberName == 1 and CountToReduceSamples < (4 * i)) or (NumberName == 2 and CountToReduceSamples < (6 * i)) or (NumberName == 3 and CountToReduceSamples < (8 * i))):
                if (os.path.isdir("CVFolders/%s/%d/Sample-%s-%02d%03d" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1)))):
                    #print("CVFolders/%s/%d/Sample-%s-%02d%03d" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1)))
                    FilesList = sorted_aphanumeric(os.listdir("CVFolders/%s/%d/Sample-%s-%02d%03d" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1)))) # Ordena archivos en directorio
                    list_im = ["CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[0]),
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[1]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[2]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[3]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[4]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[5]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[6]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[7]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[8]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[9]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[10]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[11]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[12]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[13]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[14]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[15]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[16]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[17]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[18]), 
                               "CVFolders/%s/%d/Sample-%s-%02d%03d/%s" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1), FilesList[19]),]

                    imgs = [ PIL.Image.open(i) for i in list_im ]
                    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
                    min_shape = sorted([(np.sum(i.size), i.size ) for i in imgs])[0][1]
                    imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

                    # save that beautiful picture
                    imgs_comb = PIL.Image.fromarray( imgs_comb)
                    if Folds == 0:
                        imgs_comb.save("CrossValidationFolderSmallRedim/%s/Test_folder/Sample-%s-%02d%03d.jpg" % (TestTrainValid[Folds], ShortName[NumberName], Number, (SampleNumber+1)))   
                    else:
                        imgs_comb.save("CrossValidationFolderSmallRedim/%s/%d/Sample-%s-%02d%03d.jpg" % (TestTrainValid[Folds], Number, ShortName[NumberName], Number, (SampleNumber+1)))
                    CountToReduceSamples += 1

### Cross Validation con Keras

Acá se buscan los parametros necesarios para hacer correr el entrenamiento.

Pagina guía:

https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720

Paginas de ayuda:

https://keras.io/layers/core/#flatten

https://keras.io/getting-started/sequential-model-guide/

https://keras.io/models/sequential/

### Generador de Train, Valid, Test

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(
    directory=r"/content/gdrive/My Drive/2019/Sistemas Inteligentes/Proyecto Air Writing Grupo 4/Entrega 2/DataSet/CrossValidationFolderSmallRedim/Train/",
    target_size=(1280, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
'''
The directory must be set to the path where your ‘n’ classes of folders are present.
The target_size is the size of your input images, every image will be resized to this size.
color_mode: if the image is either black and white or grayscale set “grayscale” or if the image has three colour channels, set “rgb”.
batch_size: No. of images to be yielded from the generator per batch.
class_mode: Set “binary” if you have only two classes to predict, if not set to“categorical”, in case if you’re developing an Autoencoder system, both input and the output would probably be the same image, for this case set to “input”.
shuffle: Set True if you want to shuffle the order of the image that is being yielded, else set False.
seed: Random seed for applying random image augmentation and shuffling the order of the image.
'''
valid_generator = valid_datagen.flow_from_directory(
    directory=r"/content/gdrive/My Drive/2019/Sistemas Inteligentes/Proyecto Air Writing Grupo 4/Entrega 2/DataSet/CrossValidationFolderSmallRedim/Valid/",
    target_size=(1280, 64),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

'''Same as train generator settings except for obvious changes like directory path.'''

test_generator = test_datagen.flow_from_directory(
    directory=r"/content/gdrive/My Drive/2019/Sistemas Inteligentes/Proyecto Air Writing Grupo 4/Entrega 2/DataSet/CrossValidationFolderSmallRedim/Test/",
    target_size=(1280, 64),
    color_mode="grayscale",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)
'''directory: path where there exists a folder, under which all the test images are present. For example, in this case, the images are found in /test/test_images/
batch_size: Set this to some number that divides your total number of images in your test set exactly.
Why this only for test_generator?
Actually, you should set the “batch_size” in both train and valid generators to some number that divides your total number of images in your train set and valid respectively, but this doesn’t matter before because even if batch_size doesn’t match the number of samples in the train or valid sets and some images gets missed out every time we yield the images from generator, but it would be sampled the very next epoch you train.
But for the test set, you should sample the images exactly once, no less or no more. If Confusing, just set it to 1(but maybe a little bit slower).
class_mode: Set this to None, to return only the images.
shuffle: Set this to False, because you need to yield the images in “order”, to predict the outputs and match them with their unique ids or filenames.'''



Found 264 images belonging to 11 classes.
Found 88 images belonging to 11 classes.
Found 88 images belonging to 1 classes.


'directory: path where there exists a folder, under which all the test images are present. For example, in this case, the images are found in /test/test_images/\nbatch_size: Set this to some number that divides your total number of images in your test set exactly.\nWhy this only for test_generator?\nActually, you should set the “batch_size” in both train and valid generators to some number that divides your total number of images in your train set and valid respectively, but this doesn’t matter before because even if batch_size doesn’t match the number of samples in the train or valid sets and some images gets missed out every time we yield the images from generator, but it would be sampled the very next epoch you train.\nBut for the test set, you should sample the images exactly once, no less or no more. If Confusing, just set it to 1(but maybe a little bit slower).\nclass_mode: Set this to None, to return only the images.\nshuffle: Set this to False, because you need to yield the ima

### Prueba de distintos modelos de entrenamiento
Se realizó una gran cantidad de pruebas modificando los distintos parametros, llegando a lo siguiente

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras import regularizers, optimizers
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D



'''from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np'''


model = Sequential()

#layer1
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(1280,64,1)))
model.add(Activation('sigmoid'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Layer2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


#layer Output
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="SGD"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adagrad"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adadelta"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
#model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adamax"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Nadam"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)

Epoch 1/20
8/8 [==============================] - 5s 630ms/step - loss: 6.7403 - acc: 0.1131 - val_loss: 2.3984 - val_acc: 0.0714
Epoch 2/20
8/8 [==============================] - 2s 195ms/step - loss: 2.3973 - acc: 0.0977 - val_loss: 2.3953 - val_acc: 0.0893
Epoch 3/20
8/8 [==============================] - 1s 181ms/step - loss: 2.4545 - acc: 0.1052 - val_loss: 2.3969 - val_acc: 0.1094
Epoch 4/20
8/8 [==============================] - 1s 177ms/step - loss: 2.4892 - acc: 0.1092 - val_loss: 2.3961 - val_acc: 0.0893
Epoch 5/20
8/8 [==============================] - 1s 178ms/step - loss: 2.4078 - acc: 0.1210 - val_loss: 2.3880 - val_acc: 0.2143
Epoch 6/20
8/8 [==============================] - 1s 181ms/step - loss: 2.3229 - acc: 0.2130 - val_loss: 2.2630 - val_acc: 0.3281
Epoch 7/20
8/8 [==============================] - 1s 179ms/step - loss: 2.2336 - acc: 0.2170 - val_loss: 1.8807 - val_acc: 0.2857
Epoch 8/20
8/8 [==============================] - 1s 180ms/step - loss: 2.1568 - acc: 0.24

Después de lo anterior, de notó que en muchas epochs se llegaba a un resultado cercano a 1 pero no llegaba a la solución, por lo que se decidió disminuir el valor de learning rate (lr) un poco para que así recorra una menor distancia entre cada epoch y de esa manera no se pase demasiado del resultado deseado.

* Disminucion de 0.001 a 0.0007

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras import regularizers, optimizers
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D



'''from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np'''


model = Sequential()

#layer1
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(1280,64,1)))
model.add(Activation('sigmoid'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Layer2
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


#layer Output
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0007, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="SGD"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adagrad"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adadelta"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adam"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adamax"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Nadam"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)

Epoch 1/20
8/8 [==============================] - 5s 648ms/step - loss: 12.7035 - acc: 0.0830 - val_loss: 13.4848 - val_acc: 0.0357
Epoch 2/20
8/8 [==============================] - 2s 193ms/step - loss: 6.3582 - acc: 0.0977 - val_loss: 2.3930 - val_acc: 0.1786
Epoch 3/20
8/8 [==============================] - 1s 180ms/step - loss: 2.4002 - acc: 0.0948 - val_loss: 2.3908 - val_acc: 0.1875
Epoch 4/20
8/8 [==============================] - 1s 180ms/step - loss: 2.3776 - acc: 0.1394 - val_loss: 2.3771 - val_acc: 0.1607
Epoch 5/20
8/8 [==============================] - 1s 180ms/step - loss: 2.3745 - acc: 0.1422 - val_loss: 2.3639 - val_acc: 0.1964
Epoch 6/20
8/8 [==============================] - 1s 181ms/step - loss: 2.2774 - acc: 0.1670 - val_loss: 2.2872 - val_acc: 0.2031
Epoch 7/20
8/8 [==============================] - 1s 180ms/step - loss: 2.2696 - acc: 0.1566 - val_loss: 1.9854 - val_acc: 0.3929
Epoch 8/20
8/8 [==============================] - 1s 181ms/step - loss: 1.8947 - acc: 0.

Cabe destacar que los resultados fueron tomados con una baja cantidad de mustras, lo que permitío hacer una gran cantidad de pruebas en poco tiempo.

Para usar la version con todas la muestras se puede utilizatr el siguiente directorio:
* directory=r"/content/gdrive/My Drive/2019/Sistemas Inteligentes/Proyecto Air Writing Grupo 4/Entrega 2/DataSet/CrossValidationFolderRedim/Test/"

---

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

[0.33857804962566923, 0.8928571513720921]

## Red neuronal normal con 3 capas

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras import regularizers, optimizers
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D



'''from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np'''


model = Sequential()
model.add(Conv2D(3, (3, 3), padding='same',
                 input_shape=(1280,64,1)))
model.add(Activation('relu'))
model.add(Conv2D(3, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(3, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(3, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(3))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(11, activation='softmax'))
model.compile(optimizer='rmsprop',loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="SGD"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adagrad"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adadelta"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adam"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Adamax"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
# model.compile(optimizer="Nadam"(lr=0.001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])


STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20
)

Epoch 1/20
8/8 [==============================] - 5s 566ms/step - loss: 7.9853 - acc: 0.0869 - val_loss: 13.5277 - val_acc: 0.1607
Epoch 2/20
8/8 [==============================] - 0s 37ms/step - loss: 8.7358 - acc: 0.0664 - val_loss: 14.9668 - val_acc: 0.0714
Epoch 3/20
8/8 [==============================] - 0s 52ms/step - loss: 8.9762 - acc: 0.0855 - val_loss: 14.3552 - val_acc: 0.1094
Epoch 4/20
8/8 [==============================] - 1s 78ms/step - loss: 7.8281 - acc: 0.0909 - val_loss: 15.2546 - val_acc: 0.0536
Epoch 5/20
8/8 [==============================] - 1s 79ms/step - loss: 9.2135 - acc: 0.0830 - val_loss: 14.3912 - val_acc: 0.1071
Epoch 6/20
8/8 [==============================] - 1s 85ms/step - loss: 8.9464 - acc: 0.0672 - val_loss: 14.1033 - val_acc: 0.1250
Epoch 7/20
8/8 [==============================] - 1s 78ms/step - loss: 8.0552 - acc: 0.0869 - val_loss: 15.5424 - val_acc: 0.0357
Epoch 8/20
8/8 [==============================] - 1s 80ms/step - loss: 9.1784 - acc: 0.07

### Códigosde ejemplo rescatados de internet
from https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/

In [ ]:

# initialize our Keras model and compile it
model = MiniVGGNet.build(64, 64, 3, len(lb.classes_))
opt = SGD(lr=1e-2, momentum=0.9, decay=1e-2 / NUM_EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
 
# train the network
print("[INFO] training w/ generator...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=NUM_TRAIN_IMAGES // BS,
	validation_data=testGen,
	validation_steps=NUM_TEST_IMAGES // BS,
	epochs=NUM_EPOCHS)

### Multilayer Perceptron (MLP) for multi-class softmax classification:

Ejemplo encontrado que es el cual es el que se intenta seguir como ejemplo para realizar el entrenamiento

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

Epoch 1/20
1000/1000 [==============================] - 0s 472us/step - loss: 2.4573 - acc: 0.0940
Epoch 2/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.4022 - acc: 0.0950
Epoch 3/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.3552 - acc: 0.1020
Epoch 4/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.3251 - acc: 0.1230
Epoch 5/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.3349 - acc: 0.1040
Epoch 6/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.3266 - acc: 0.0970
Epoch 7/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.3160 - acc: 0.1130
Epoch 8/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.2973 - acc: 0.1250
Epoch 9/20
1000/1000 [==============================] - 0s 24us/step - loss: 2.3057 - acc: 0.1120
Epoch 10/20
1000/1000 [==============================] - 0s 23us/step - loss: 2.3105 - acc: 0.1120
Epoch 11/20
1000/1